Fall Block

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score

def preprocess_data(spring_df):
    X = spring_df.drop(columns=['StudentID','Years of Work Experience'])  # Exclude 'StudentID' from feature processing
    median_value = spring_df['Official Work Experience'].median()
    X['Official Work Experience'].fillna(median_value, inplace=True)
    X['Employer'].fillna('NA', inplace=True)
    X['Industry'].fillna('NA', inplace=True)
    X.dropna(inplace=True)
    spring_df2 = X.copy()
    le = LabelEncoder()
    for col in spring_df2.columns:
        if spring_df2[col].dtype == 'object':
            spring_df2[col] = le.fit_transform(spring_df2[col])
    spring_df2['StudentID'] = spring_df['StudentID']
    return spring_df2

def create_diverse_group(cluster, group_size_range):
    group = []
    while len(group) < group_size_range[1] and len(cluster) > 0:
        if len(group) == 0:
            new_member_idx = np.random.choice(cluster.index)
        else:
            # Exclude 'StudentID' from distance calculation
            group_data = np.array([item[:-1] for item in group])
            cluster_data = cluster.drop(columns=['StudentID']).to_numpy()
            distances = cdist(group_data, cluster_data)
            min_distances = distances.min(axis=0)
            new_member_idx = min_distances.argmax()

        new_member = cluster.iloc[new_member_idx]
        group.append(new_member.tolist())
        cluster = cluster.drop(cluster.index[new_member_idx]).reset_index(drop=True)

    return group, cluster

def distribute_points_among_groups(df, num_cohorts, cohort_size):
    all_groups = []
    while not df.empty:
        group, df = create_diverse_group(df, (4, cohort_size))
        all_groups.append(group)

    cohorts = []
    for _ in range(num_cohorts):
        current_cohort = []
        while len(current_cohort) < cohort_size and all_groups:
            current_cohort.extend(all_groups.pop(0))
            if len(current_cohort) > cohort_size:
                all_groups.insert(0, current_cohort[cohort_size:])
                current_cohort = current_cohort[:cohort_size]
        cohorts.append(current_cohort)

    return cohorts

def combine_cohorts_into_df(cohorts):
    combined_df = pd.DataFrame()
    for index, cohort in enumerate(cohorts):
        cohort_df = pd.DataFrame(cohort, columns=['Gender', 'UG School', 'UG Major', 'Employer', 'Industry', 'Official Work Experience', 'Citizenship Country Code', 'StudentID'])
        cohort_df.insert(0, 'Cohort', index + 1)
        combined_df = pd.concat([combined_df, cohort_df], ignore_index=True)
    return combined_df[['StudentID', 'Cohort']]

def assign_classes_to_cohorts(result_df, num_classes):
    total_cohorts = result_df['Cohort'].nunique()
    np.random.seed(0)
    cohorts = result_df['Cohort'].unique()
    np.random.shuffle(cohorts)
    classes = np.array_split(cohorts, num_classes)
    cohort_to_class = {cohort: i + 1 for i, cohort_list in enumerate(classes) for cohort in cohort_list}
    result_df['Class'] = result_df['Cohort'].map(cohort_to_class)
    result_df.rename(columns={'Cohort': 'Team', 'Class': 'Group'}, inplace=True)
    return result_df

def calculate_silhouette_scores(df, label_column):
    features = df.drop(columns=['Team','Group','StudentID'], errors='ignore')
    labels = df[label_column]
    if labels.isna().any():
        print("Warning: NaN found in label data")
        return np.nan  # Return NaN score if any NaNs are present in labels
    return silhouette_score(features, labels)

def main(spring_df, cohort_size, num_classes):
    spring_df2 = preprocess_data(spring_df)
    #print(spring_df2)
    num_cohorts = len(spring_df2) // cohort_size
    cohorts = distribute_points_among_groups(spring_df2, num_cohorts, cohort_size)
    combined_df = combine_cohorts_into_df(cohorts)
    result_df = assign_classes_to_cohorts(combined_df, num_classes)
    sil_df = pd.merge(spring_df2, result_df, on='StudentID', how='left')
    #performance scores
    cohort_score = calculate_silhouette_scores(sil_df.dropna(), 'Team')
    class_score = calculate_silhouette_scores(sil_df.dropna(), 'Group')
    ##################
    final_df = pd.merge(spring_df, result_df, on='StudentID', how='left')
    final_df.drop(columns=['Years of Work Experience'],inplace=True)
    return final_df, cohort_score, class_score

# Example usage
fall_df = pd.read_csv('year2_roster_data.csv')  # Replace with actual path
cohort_size = 6
num_classes = 5
#final_fall_df = main(fall_df, cohort_size, num_classes)
final_fall_df, cohort_score, class_score = main(fall_df, cohort_size, num_classes)
print(final_fall_df.head())
print("Silhouette Score by Cohort:", cohort_score)
print("Silhouette Score by Class:", class_score)

   StudentID  Gender                                  UG School  \
0          1    Male  Motilal Nehru National Institute of Techn   
1          2  Female                        University Of Tulsa   
2          3    Male                    The NorthCap University   
3          4  Female                       University of Madras   
4          5    Male  Kwame Nkrumah University of Science and T   

                         UG Major      Employer           Industry  \
0         Electronics & Comm Engg          Noon             Retail   
1  Manangement Information System  Saudi Aramco  Business Services   
2  Computer Science & Engineering           NaN                NaN   
3                     Mathematics           NaN                NaN   
4         Business Administration           NaN                NaN   

   Official Work Experience Citizenship Country Code  Team  Group  
0                       7.0                      IND  31.0    1.0  
1                       9.0             

In [ ]:
final_fall_df.to_csv('Fall_List.csv')

In [ ]:
import pandas as pd

df_cohort_score = pd.DataFrame({'Cohort Score': [cohort_score]})
df_class_score = pd.DataFrame({'Class Score': [class_score]})
# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter('final_fall_data.xlsx', engine='xlsxwriter') as writer:
    # Write each DataFrame to a different worksheet
    final_fall_df.to_excel(writer, sheet_name='Fall Data', index=False)
    df_cohort_score = pd.DataFrame({'Cohort Score': [cohort_score]})
    df_class_score = pd.DataFrame({'Class Score': [class_score]})

Adding dummy Track and Specialization Data

In [ ]:
final_fall_df['TrackLength'] = np.random.choice([9, 16], size=len(final_fall_df), p=[0.8, 0.2])
labels = ["SupplyChain", "Marketing", "CloudComputing", "Finance"]
probabilities = [0.6, 0.1, 0.15, 0.15]

# Assign labels conditionally
final_fall_df['Label'] = np.where(
    final_fall_df['TrackLength'] == 16,
    np.random.choice(labels, size=len(final_fall_df), p=probabilities),
    None  # Assign None or any other placeholder if TrackLength is not 16
)
final_fall_df.head()

,StudentID,Gender,UG School,UG Major,Employer,Industry,Official Work Experience,Citizenship Country Code,Team,Group,TrackLength,Label
0,1,Male,Motilal Nehru National Institute of Techn,Electronics & Comm Engg,Noon,Retail,7.0,IND,31.0,1.0,9,None
1,2,Female,University Of Tulsa,Manangement Information System,Saudi Aramco,Business Services,9.0,SAU,24.0,5.0,9,None
2,3,Male,The NorthCap University,Computer Science & Engineering,NaN,NaN,0.0,IND,32.0,2.0,9,None
3,4,Female,University of Madras,Mathematics,NaN,NaN,3.0,IND,28.0,1.0,9,None
4,5,Male,Kwame Nkrumah University of Science and T,Business Administration,NaN,NaN,0.0,GHA,22.0,5.0,9,None


In [ ]:
final_fall_df.to_csv('Spring_Input_List.csv')

Spring Block

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import cdist

def preprocess_data(spring_df):
    spring_df['Employer'].fillna('NA', inplace=True)
    spring_df['Industry'].fillna('NA', inplace=True)
    spring_df['Label'].fillna('NA', inplace=True)
    spring_df.dropna(inplace=True)
    spring_df2 = spring_df.copy()
    spring_df2.drop(['Group', 'Team', 'StudentID', 'TrackLength', 'Label'], axis=1, inplace=True)

    le = LabelEncoder()
    for col in spring_df2.columns:
        if spring_df2[col].dtype == 'object':
            spring_df2[col] = le.fit_transform(spring_df2[col])

    #spring_df2['cluster'] = spring_df['cluster']
    spring_df2['Group'] = spring_df['Group']
    spring_df2['Team'] = spring_df['Team']
    spring_df2['StudentID'] = spring_df['StudentID']
    spring_df2['TrackLength'] = spring_df['TrackLength']
    spring_df2['Label'] = spring_df['Label']

    return spring_df2

def create_diverse_group(cluster, current_teams, group_size_range):
    group = []
    while len(group) < group_size_range[1] and len(cluster) > 0:
        if len(group) == 0:
            new_member_idx = np.random.choice(cluster.index)
        else:
            group_data = np.array([member[:-6] for member in group])
            distances = cdist(group_data, cluster.iloc[:, :-6])
            min_distances = distances.min(axis=0)
            if not min_distances.size:
                break
            new_member_idx = min_distances.argmax()

        new_member = cluster.iloc[new_member_idx]
        if new_member['Team'] in current_teams:
            cluster = cluster.drop(cluster.index[new_member_idx]).reset_index(drop=True)
            continue
        group.append(new_member.values.tolist())
        current_teams.add(new_member['Team'])
        cluster = cluster.drop(cluster.index[new_member_idx]).reset_index(drop=True)

    return group, cluster

def distribute_points_among_groups(df, num_cohorts, cohort_size):
    df_9 = df[df['TrackLength'] == 9].reset_index(drop=True)
    df_16 = df[df['TrackLength'] == 16].reset_index(drop=True)

    all_groups = []
    track_labels = df_16['Label'].unique()

    while not df_9.empty:
        group, df_9 = create_diverse_group(df_9, set(), (4, 5))
        all_groups.append(group)

    for label in track_labels:
        df_label = df_16[df_16['Label'] == label].reset_index(drop=True)
        while not df_label.empty:
            group, df_label = create_diverse_group(df_label, set(), (4, 5))
            all_groups.append(group)

    cohorts = []
    for _ in range(num_cohorts):
        current_cohort = []
        while len(current_cohort) < cohort_size and all_groups:
            current_cohort.extend(all_groups.pop(0))
            if len(current_cohort) > cohort_size:
                all_groups.insert(0, current_cohort[cohort_size:])
                current_cohort = current_cohort[:cohort_size]
        cohorts.append(current_cohort)

    if len(cohorts[-1]) < 4:
        last_cohort = cohorts.pop(-1)
        for student in last_cohort:
            for cohort in cohorts:
                if len(cohort) < cohort_size:
                    cohort.append(student)
                    break

    return cohorts

def combine_cohorts_into_df(cohorts):
    columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M']
    combined_df = pd.DataFrame(columns=['Cohort'] + columns)
    for index, cohort in enumerate(cohorts):
        cohort_df = pd.DataFrame(cohort, columns=columns)
        cohort_df.insert(0, 'Cohort', index + 1)
        combined_df = pd.concat([combined_df, cohort_df], ignore_index=True)

    combined_df_renamed = combined_df.rename(columns={
        'A': 'Gender',
        'B': 'UG School',
        'C': 'UG Major',
        'D': 'Employer',
        'E': 'Industry',
        'F': 'Official Work Experience',
        'G': 'Citizenship Country Code',
        'H': 'Group',
        'I': 'Team',
        'K': 'StudentID',
        'L': 'TrackLength',
        'M': 'Label'
    })

    return combined_df_renamed

def assign_classes_to_cohorts(result_df, num_classes):
    total_cohorts = result_df['Cohort'].nunique()
    total_students = result_df.shape[0]

    np.random.seed(0)
    cohorts = result_df['Cohort'].unique()
    np.random.shuffle(cohorts)

    classes = np.array_split(cohorts, num_classes)
    cohort_to_class = {cohort: i + 1 for i, cohort_list in enumerate(classes) for cohort in cohort_list}

    result_df['Class'] = result_df['Cohort'].map(cohort_to_class)

    return result_df

def calculate_silhouette_scores(df, label_column):
    features = df.drop(columns=['Cohort','Class','StudentID','Group','Team','TrackLength','Label'], errors='ignore')
    labels = df[label_column]
    if labels.isna().any():
        print("Warning: NaN found in label data")
        return np.nan  # Return NaN score if any NaNs are present in labels
    return silhouette_score(features, labels)

def main(spring_df, cohort_size, num_classes):
    spring_df2 = preprocess_data(spring_df)
    num_cohorts = len(spring_df2) // cohort_size
    cohorts = distribute_points_among_groups(spring_df2, num_cohorts, cohort_size)
    combined_df_renamed = combine_cohorts_into_df(cohorts)
    # print(combined_df_renamed.head())
    #sil_df = pd.merge(spring_df2, combined_df_renamed, on='StudentID', how='left')
    #print(spring_df2.columns)
    #performance scores
    join_columns = ['Gender', 'UG School', 'UG Major', 'Employer', 'Industry', 'Official Work Experience', 'Citizenship Country Code', 'Group', 'Team', 'StudentID', 'TrackLength', 'Label']
    result_df = pd.merge(spring_df2, combined_df_renamed, left_on=join_columns, right_on=join_columns, how='left')
    result_df = assign_classes_to_cohorts(result_df, num_classes)

    sil_df = pd.merge(spring_df2, result_df[['StudentID', 'Cohort', 'Class']], on='StudentID', how='left')
    #Scores
    cohort_score = calculate_silhouette_scores(sil_df.dropna(), 'Cohort')
    class_score = calculate_silhouette_scores(sil_df.dropna(), 'Class')
    #Output
    result_df2 = pd.merge(spring_df, result_df[['StudentID', 'Cohort', 'Class']], on='StudentID', how='left')

    return result_df2, cohort_score, class_score

# Example usage
#spring_df = pd.read_csv('path_to_spring_df.csv')  # Replace with actual path
cohort_size = 6
num_classes = 5
spring_df = final_fall_df.copy()
final_spring_df, cohort_score, class_score = main(spring_df, cohort_size, num_classes)
print(final_spring_df.head())
print("Silhouette Score by Team:", cohort_score)
print("Silhouette Score by Cohort:", class_score)
# Save the final DataFrame to a CSV file
# final_df.to_csv('final_df.csv', index=False)

  StudentID  Gender                                  UG School  \
0         1    Male  Motilal Nehru National Institute of Techn   
1         2  Female                        University Of Tulsa   
2         3    Male                    The NorthCap University   
3         4  Female                       University of Madras   
4         5    Male  Kwame Nkrumah University of Science and T   

                         UG Major      Employer           Industry  \
0         Electronics & Comm Engg          Noon             Retail   
1  Manangement Information System  Saudi Aramco  Business Services   
2  Computer Science & Engineering            NA                 NA   
3                     Mathematics            NA                 NA   
4         Business Administration            NA                 NA   

   Official Work Experience Citizenship Country Code  Team  Group  \
0                       7.0                      IND  31.0    1.0   
1                       9.0                 

In [ ]:
pip install xlsxwriter

In [ ]:
import pandas as pd

df_cohort_score = pd.DataFrame({'Cohort Score': [cohort_score]})
df_class_score = pd.DataFrame({'Class Score': [class_score]})
# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter('final_spring_data.xlsx', engine='xlsxwriter') as writer:
    # Write each DataFrame to a different worksheet
    final_spring_df.to_excel(writer, sheet_name='Spring Data', index=False)
    df_cohort_score = pd.DataFrame({'Cohort Score': [cohort_score]})
    df_class_score = pd.DataFrame({'Class Score': [class_score]})